In [52]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [54]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [55]:
# setup
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import time
import datetime as dt
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import scipy.spatial.distance as dist

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [56]:
def print_tracks(uris):
    #recurse through all uris
    if len(uris) > 50:
        num_sets = int(len(uris)/50)
        for i in range(num_sets):
            print_tracks(uris[50*i:50*(i+1)])
        if len(uris)%50 != 0:
            print_tracks(uris[50*num_sets:])
        return

    # collect chosen tracks
    chosen_tracks = sp.tracks(uris)
    chosen_tracks_dict = pd.DataFrame(chosen_tracks['tracks'])
    chosen_tracks_names = chosen_tracks_dict['name']
    chosen_artists_dict = chosen_tracks_dict['artists']
    chosen_artists = []
    for song in chosen_artists_dict:
        a = []
        for artist in song:
            a.append(artist['name'])
        chosen_artists.append(a)

    # print chosen tracks
    for i in range(len(chosen_tracks_names)):
        chosen_artists_string = ''
        for j in range(len(chosen_artists[i])):
            # print(artist)
            chosen_artists_string += chosen_artists[i][j]
            if j < len(chosen_artists[i])-1:
                chosen_artists_string += ', '
        print(chosen_tracks_names[i] + ' by ' + chosen_artists_string)
    
    # print(pd.DataFrame(np.array([np.array(chosen_tracks_names), chosen_artists]).T))

In [57]:
NUM_RECOMMENDATIONS = 100
def get_recommendations(uris):
    recommendations_uris = pd.DataFrame(sp.recommendations(seed_tracks=uris, limit=NUM_RECOMMENDATIONS)['tracks'])['uri']
    return sp.audio_features(recommendations_uris)

In [63]:
MS_IN_DAY = 86400
# get artist, artist genres, popularity, date
def get_track_values(uris):
    # return values are messed up - gotta pass through the return values!!
    extra_uris = []
    if len(uris) > 50:
        extra_uris = uris[50:]
        uris = uris[:50]

    # collect chosen tracks
    tracks = sp.tracks(uris)
    tracks_dict = pd.DataFrame(tracks['tracks'])
    names = tracks_dict['name']
    
    popularity = list(tracks_dict['popularity'])

    artists_dict = tracks_dict['artists']
    artists = []
    for song in artists_dict:
        a = []
        for artist in song:
            a.append(artist['name'])
        artists.append(a)

    # get genres
    # flatten artists list, make sure len < 50, get genres - prob separate method

    albums_dict = pd.DataFrame(tracks_dict['album'].to_list())
    dates = albums_dict['release_date']
    # dates = []
    # datetime conversion
    # for date in unconverted_dates:
    #     while len(date) < 10:
    #         date += '-01'
    #     ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
    #     dates.append(ms_date / MS_IN_DAY)
    
    track_values = [artists, popularity, dates, names]
    
    if len(extra_uris) > 0:
        recur_values = get_track_values(extra_uris)
        for i in range(len(track_values)):
            track_values[i].extend(recur_values[i])
    return track_values

In [69]:
# function to get audio features, gkde, and scaler from a set of 1-5 URIs
def get_features_without_recs(uris):
    # Get audio features for user's chosen tracks from Spotify API
    chosen_features = sp.audio_features(uris)
    
    # Create unweighted list of all features
    features = pd.DataFrame(chosen_features)
    features = features.drop(['key', 'loudness', 'mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis=1)
    # Scale features to have mean 0 and var 1
    artists, popularity, dates, names = get_track_values(uris)
    features['popularity'] = popularity
    features['date'] = dates
    features['artists'] = artists
    features.insert(loc=0, column='names', value=names)
    
    return features

In [73]:
# Ain't No Mountain High Enough, Me Porto Bonito, Animal, WHOLE LOTTA MONEY, Goodies, Griztronics II, Mozart, Up On Cripple Creek Live, Latch Acoustic, Down Jay Sean
test_metrics_uris = ['7tqhbajSfrz2F7E1Z75ASX', '6Sq7ltF9Qa7SNFBsV5Cogx', '1fBl642IhJOE5U319Gy2Go', '67G6iaOw8DZqp1z8STR89R', '1rGd2CoMxEg2dnaytLY5ZG', '2il9NDpW87Hk2L9AjeDf8r', '429GaRIWKJ3EU1GAQPbLcr', '1CBR6p0Hv8UA21l0z5Z6Tw', '5vUNRJsvEOUsz4sbKkJpFr', '6cmm1LMvZdB5zsCwX5BjqE']
get_features_without_recs(test_metrics_uris)

,names,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,date,artists
0,Ain't No Mountain High Enough,0.663,0.6000,0.0320,0.430000,0.000000,0.1840,0.800,129.991,81,1967-08-29,"[Marvin Gaye, Tammi Terrell]"
1,Me Porto Bonito,0.911,0.7120,0.0817,0.090100,0.000027,0.0933,0.425,92.005,96,2022-05-06,"[Bad Bunny, Chencho Corleone]"
2,Animal,0.482,0.8330,0.0449,0.000346,0.000000,0.3650,0.740,148.039,70,2010-01-01,[Neon Trees]
3,WHOLE LOTTA MONEY (feat. Nicki Minaj) - Remix,0.893,0.4630,0.3600,0.199000,0.000000,0.3180,0.591,81.026,68,2021-07-09,"[BIA, Nicki Minaj]"
4,Goodies,0.844,0.8650,0.0431,0.019600,0.009790,0.0978,0.679,126.008,64,2022-09-02,[Dillon Francis]
5,Griztronics II (Another Level),0.579,0.9780,0.2110,0.020500,0.008290,0.3260,0.580,145.054,55,2021-11-19,"[GRiZ, Subtronics]"
6,"Piano Concerto No. 21 in C Major, K. 467: II. ...",0.137,0.0311,0.0381,0.982000,0.972000,0.0908,0.063,179.223,62,1995-10-01,"[Wolfgang Amadeus Mozart, London Mozart Player..."
7,"Up on Cripple Creek - Live at LA Forum, Inglew...",0.562,0.9160,0.1380,0.178000,0.000000,0.9840,0.634,136.369,21,1974-06-20,"[Bob Dylan, The Band]"
8,Latch - Acoustic,0.372,0.2470,0.0273,0.981000,0.000223,0.1100,0.239,96.170,64,2015-11-06,[Sam Smith]
9,Down,0.727,0.6800,0.0286,0.010100,0.000000,0.0821,0.728,132.012,77,2009-01-01,"[Jay Sean, Lil Wayne]"


In [72]:
# Me Porto Bonito, Animal, WHOLE LOTTA MONEY, Goodies, Griztronics II, Mozart
test_metrics_uris = ['6Sq7ltF9Qa7SNFBsV5Cogx', '1fBl642IhJOE5U319Gy2Go', '67G6iaOw8DZqp1z8STR89R', '1rGd2CoMxEg2dnaytLY5ZG', '2il9NDpW87Hk2L9AjeDf8r', '429GaRIWKJ3EU1GAQPbLcr']
get_features_without_recs(test_metrics_uris)

,names,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,date,artists
0,Me Porto Bonito,0.911,0.7120,0.0817,0.090100,0.000027,0.0933,0.425,92.005,96,2022-05-06,"[Bad Bunny, Chencho Corleone]"
1,Animal,0.482,0.8330,0.0449,0.000346,0.000000,0.3650,0.740,148.039,70,2010-01-01,[Neon Trees]
2,WHOLE LOTTA MONEY (feat. Nicki Minaj) - Remix,0.893,0.4630,0.3600,0.199000,0.000000,0.3180,0.591,81.026,68,2021-07-09,"[BIA, Nicki Minaj]"
3,Goodies,0.844,0.8650,0.0431,0.019600,0.009790,0.0978,0.679,126.008,64,2022-09-02,[Dillon Francis]
4,Griztronics II (Another Level),0.579,0.9780,0.2110,0.020500,0.008290,0.3260,0.580,145.054,55,2021-11-19,"[GRiZ, Subtronics]"
5,"Piano Concerto No. 21 in C Major, K. 467: II. ...",0.137,0.0311,0.0381,0.982000,0.972000,0.0908,0.063,179.223,62,1995-10-01,"[Wolfgang Amadeus Mozart, London Mozart Player..."


Notes

Danceability:
- Me Porto Bonito is higher than I thought it would be

Energy feels accurate
- WHOLE LOTTA MONEY is low, Griztronics II is high

Speechiness is accurate
- only value above .33 threshold is the rap song

Low acousticness, instrumentalness, liveness (except mozart)

Valence makes sense: "musical positiveness"
- Animal def should be highest on this list

Popularity also makes sense

In [66]:
# Animal, Griztronics II, Let's Get Weird, Up On Cripple Creek Live
test_metrics_uris = ['1fBl642IhJOE5U319Gy2Go', '2il9NDpW87Hk2L9AjeDf8r', '0NQVIXbk5AIHzNrBdlcTP9', '1CBR6p0Hv8UA21l0z5Z6Tw']
get_features_without_recs(test_metrics_uris)

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,date,artists,names
0,0.482,0.833,0.0449,0.000346,0.00000,0.365,0.740,148.039,70,2010-01-01,[Neon Trees],Animal
1,0.579,0.978,0.2110,0.020500,0.00829,0.326,0.580,145.054,55,2021-11-19,"[GRiZ, Subtronics]",Griztronics II (Another Level)
2,0.719,0.891,0.2810,0.006370,0.03120,0.814,0.595,146.291,31,2019-11-15,[GRiZ],Let's Get Weird
3,0.562,0.916,0.1380,0.178000,0.00000,0.984,0.634,136.369,21,1974-06-20,"[Bob Dylan, The Band]","Up on Cripple Creek - Live at LA Forum, Inglew..."


Interesting observation: rock tracks and dubstep tracks have similar values
Also, let's get weird's liveness is just incorrect

In [67]:
# Mozart, Up On Cripple Creek Live, Latch Acoustic
test_metrics_uris = ['429GaRIWKJ3EU1GAQPbLcr', '1CBR6p0Hv8UA21l0z5Z6Tw', '5vUNRJsvEOUsz4sbKkJpFr']
get_features_without_recs(test_metrics_uris)

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,date,artists,names
0,0.137,0.0311,0.0381,0.982,0.972000,0.0908,0.063,179.223,62,1995-10-01,"[Wolfgang Amadeus Mozart, London Mozart Player...","Piano Concerto No. 21 in C Major, K. 467: II. ..."
1,0.562,0.9160,0.1380,0.178,0.000000,0.9840,0.634,136.369,21,1974-06-20,"[Bob Dylan, The Band]","Up on Cripple Creek - Live at LA Forum, Inglew..."
2,0.372,0.2470,0.0273,0.981,0.000223,0.1100,0.239,96.170,64,2015-11-06,[Sam Smith],Latch - Acoustic


Mozart - high acousticness and instrumentalness, low liveness
Up On Cripple Creek (Live) - low acousticness and instrumentalness, high liveness
Latch (Acoustic) - high acousticness, low instrumentalness and liveness

Barbell variables - how do I include these?
Acousticness - no threshold
Instrumentalness threshold = > 0.5 is instrumental
Liveness threshold = > 0.8 is live

Notes:
Danceability, energy, speechiness, acousticness, instrumentalness generally map as expected
- surprising that Me Porto Bonito has such high danceability
- interesting that the Rina track is almost as high energy as the GRiZ songs
- instrumentalness is almost an indicator variable - ignore?
Let's Get Weird liveness is just wrong
- >.8 is supposed to indicate that the track is live, but it is def not
Tempo: the Jay Sean track is half-timed by Spotify
- ignore? double?
Including the latin tracks makes the popularity seem even more important

In [92]:
# Ain't No Mountain High Enough, Down Jay Sean
test_metrics_uris = ['7tqhbajSfrz2F7E1Z75ASX', '6cmm1LMvZdB5zsCwX5BjqE']
get_features_without_recs(test_metrics_uris)

,names,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,date,artists
0,Ain't No Mountain High Enough,0.663,0.60,0.0320,0.4300,0,0.1840,0.800,129.991,81,1967-08-29,"[Marvin Gaye, Tammi Terrell]"
1,Down,0.727,0.68,0.0286,0.0101,0,0.0821,0.728,132.012,77,2009-01-01,"[Jay Sean, Lil Wayne]"


Very similar values except acousticness - how do we catch stuff like this and the rock/dubstep example?